#### Goal for this webscrape phase: "go into" a page with links to "zoom in"/loop through links that then take you to another level that will then loop through links to scrape each webpage with in
    Challenge:
    -there are 14 styles (categories) and around ~120 links (subcategories)
    -each style can have multiptle pages to loop through

###### Starting small by scraping one page

In [1]:
import cloudscraper
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd  
import time 
from random import randint


In [60]:
url = "https://www.beeradvocate.com/beer/profile/143/1352/" 
# url ="https://www.beeradvocate.com/beer/profile/17033/53930/"
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
scraper = cloudscraper.create_scraper() 
info = scraper.get(url) 
print(info.status_code) 
 

200


In [61]:
soup = BeautifulSoup(info.text, 'lxml') 

In [62]:
name=soup.h1.find(string=True)
name

'Shiner Bock'

In [63]:
soup.find('h1').text
#will give me both the beer title and brewery merged together

'Shiner BockSpoetzl Brewery'

In [64]:
# soup.find('dl',{'class':'beerstats'}).text

In [65]:
brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
brewery

'Spoetzl Brewery'

In [66]:
loaction=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
loaction

'Texas'

In [67]:
location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
location2

'United States'

In [68]:
style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
style

'Bock'

In [69]:
ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
ba_score
##official score on ba site 74::'okay'

'74'

In [70]:
over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
over_all_rank
##over all rank is against all beers in ba site

'Ranked #28,000'

In [71]:
style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
style_rank
## style rank is among all beers within style category

'Ranked #96'

In [72]:
alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
alcohol_percent

'4.5%'

In [27]:
avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
avg_rating
# Average across all ratings for this beer

'3.93'

In [28]:
percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
percent_dev
# Percentage of deviation within the ratings

'11.7%'

In [29]:
review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
review_count
# Number of reviews for this beer

'5'

In [30]:
rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
rating_count
# Number of ratings for this beer

'20'

In [31]:
status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
status
# the beer's current activity status

'Active'

In [32]:
date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
date_added
# The date this beer was added to the site

'Nov 05, 2009'

###### zooming out one level to scrape through "bocks" pg 1 with 50 results

In [33]:
# youtube page: John Watson Rooney helped get more of the wabscraping process 
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
website=f'{root}/beer/styles/32/?sort=revsD&start=0'
scraper = cloudscraper.create_scraper() 
info = scraper.get(website) 
print(info.status_code)


200


In [34]:
soup = BeautifulSoup(info.text, 'lxml')

In [35]:
table=soup.find('table')
# table is where the links are concentrated in 

In [36]:
table.find_all("a", href=re.compile("profile"))[::2]

[<a href="/beer/profile/143/1352/"><b>Shiner Bock</b><br/></a>,
 <a href="/beer/profile/35/14309/"><b>Samuel Adams Chocolate Bock</b><br/></a>,
 <a href="/beer/profile/29/1252/"><b>Michelob AmberBock</b><br/></a>,
 <a href="/beer/profile/35/68000/"><b>Samuel Adams Cherry Chocolate Bock</b><br/></a>,
 <a href="/beer/profile/590/2565/"><b>Uff-da</b><br/></a>,
 <a href="/beer/profile/35/74579/"><b>Samuel Adams The Vixen</b><br/></a>,
 <a href="/beer/profile/590/62671/"><b>Back Forty Bock</b><br/></a>,
 <a href="/beer/profile/125/2272/"><b>St. Nikolaus Bock Bier</b><br/></a>,
 <a href="/beer/profile/337/1070/"><b>Spring Bock</b><br/></a>,
 <a href="/beer/profile/2137/3835/"><b>Pandora's Bock</b><br/></a>,
 <a href="/beer/profile/302/41422/"><b>Genesee Spring Bock</b><br/></a>,
 <a href="/beer/profile/66/1873/"><b>Ur-Bock Dunkel</b><br/></a>,
 <a href="/beer/profile/184/20299/"><b>La Trappe Bockbier</b><br/></a>,
 <a href="/beer/profile/35/520715/"><b>Samuel Adams Winter Lager</b><br/></a>,

In [37]:
links=[link['href'] for link in table.find_all("a", href=re.compile("profile"))[::2]]
print(links)

['/beer/profile/143/1352/', '/beer/profile/35/14309/', '/beer/profile/29/1252/', '/beer/profile/35/68000/', '/beer/profile/590/2565/', '/beer/profile/35/74579/', '/beer/profile/590/62671/', '/beer/profile/125/2272/', '/beer/profile/337/1070/', '/beer/profile/2137/3835/', '/beer/profile/302/41422/', '/beer/profile/66/1873/', '/beer/profile/184/20299/', '/beer/profile/35/520715/', '/beer/profile/428/14899/', '/beer/profile/417/47795/', '/beer/profile/57/7714/', '/beer/profile/35/225611/', '/beer/profile/107/7904/', '/beer/profile/99/75739/', '/beer/profile/1819/31215/', '/beer/profile/455/2250/', '/beer/profile/15280/39884/', '/beer/profile/30128/86334/', '/beer/profile/628/5119/', '/beer/profile/220/283116/', '/beer/profile/710/507335/', '/beer/profile/1805/20968/', '/beer/profile/5502/27167/', '/beer/profile/435/6340/', '/beer/profile/410/144046/', '/beer/profile/35/289887/', '/beer/profile/31272/171485/', '/beer/profile/38015/170131/', '/beer/profile/7734/76116/', '/beer/profile/26762

In [38]:
beerlinks=[]
for link in table.find_all("a", href=re.compile("profile"))[::2]:
    beerlinks.append(root + link['href'])


In [39]:
beerlinks


['https://www.beeradvocate.com/beer/profile/143/1352/',
 'https://www.beeradvocate.com/beer/profile/35/14309/',
 'https://www.beeradvocate.com/beer/profile/29/1252/',
 'https://www.beeradvocate.com/beer/profile/35/68000/',
 'https://www.beeradvocate.com/beer/profile/590/2565/',
 'https://www.beeradvocate.com/beer/profile/35/74579/',
 'https://www.beeradvocate.com/beer/profile/590/62671/',
 'https://www.beeradvocate.com/beer/profile/125/2272/',
 'https://www.beeradvocate.com/beer/profile/337/1070/',
 'https://www.beeradvocate.com/beer/profile/2137/3835/',
 'https://www.beeradvocate.com/beer/profile/302/41422/',
 'https://www.beeradvocate.com/beer/profile/66/1873/',
 'https://www.beeradvocate.com/beer/profile/184/20299/',
 'https://www.beeradvocate.com/beer/profile/35/520715/',
 'https://www.beeradvocate.com/beer/profile/428/14899/',
 'https://www.beeradvocate.com/beer/profile/417/47795/',
 'https://www.beeradvocate.com/beer/profile/57/7714/',
 'https://www.beeradvocate.com/beer/profile/

##### Building together code to loop through links on one page  

In [40]:
# print(f'{variable im using in the for loop} : {variable 1 i am getting out of the loop this pass}, {variable 2 im getting out of the loop this pass}')
# https://medium.com/swlh/improve-your-web-scraper-with-limited-retry-loops-python-35e21730cbf5
# https://www.youtube.com/watch?v=sIOMDu6MXJQ&list=PLRzwgpycm-Fio7EyivRKOBN4D3tfQ_rpu&index=3
# https://www.youtube.com/watch?v=8dTpNajxaH0&t=503s
# https://www.freecodecamp.org/news/how-to-scrape-multiple-web-pages-using-python/

In [46]:
# testlink='https://www.beeradvocate.com/beer/profile/143/1352/'
# for page pagination, ran a range to an imposiblly high number, by 50  (results per page that url uses to fill start param)
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
website=f'{root}/beer/styles/32/?sort=revsD&start=100'
scraper = cloudscraper.create_scraper() 
response = scraper.get(website)  
soup = BeautifulSoup(response.text, 'lxml')
table=soup.find('table')
# print(table)
beerlist=[]

beerlinks=[]
for link in table.find_all("a", href=re.compile("profile"))[::2]:
    beerlinks.append(root + link['href'])
# print(beerlinks)

for link in beerlinks:
    scraper = cloudscraper.create_scraper()
    r=scraper.get(link, headers=header)
    soup=BeautifulSoup(r.text,'lxml')
    name=soup.h1.find(string=True)
    try:    
        brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
    except:
        brewery= 'Na'
    try:
        location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
    except:
        location= 'Na'
    try:
        location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
    except:
        location2= 'Na'
    try:
        style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
    except:
        style= 'Na'
    try:
        ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
    except:
        ba_score= 'Na'
    try:
        over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
    except:
        over_all_rank= 'Na'
    try:
        style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
    except:
        style_rank= 'Na'
    try:
        alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
    except:
        alcohol_percent= 'Na'
    try:
        avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
    except:
        avg_rating= 'Na'
    try:
        percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
    except:
        percent_dev= 'Na'
    try:
        review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
    except:
        review_count= 'Na'
    try:
        rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
    except:
        rating_count= 'Na'
    try:
        status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
    except:
        status= 'Na'
    try:
        date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
    except:
        date_added= 'Na'
    beer_info= {
                'name': name,
                'brewery': brewery,
                'location':location,
                'location2':location2,
                'style': style,
                'ba_score':ba_score,
                'over_all_rank': over_all_rank,
                'style_rank': style_rank,
                'alcohol_percent': alcohol_percent,
                'avg_rating':avg_rating,
                'percent_dev': percent_dev,
                'review_count': review_count,
                'rating_count':rating_count,
                'status': status,
                'date_added': date_added
                }
    beerlist.append(beer_info)
    print(f'{link}-{name}')

# print({variable 1 i am getting out of the loop this pass}, {variable 2 im getting out of the loop this pass}')

https://www.beeradvocate.com/beer/profile/16271/41963/-Bock
https://www.beeradvocate.com/beer/profile/6627/20992/-Pfungstädter St. Nikolaus
https://www.beeradvocate.com/beer/profile/24571/90388/-San Martino La Gold
https://www.beeradvocate.com/beer/profile/2304/21646/-Bibock
https://www.beeradvocate.com/beer/profile/6844/182259/-Bourbon Barrel Bock
https://www.beeradvocate.com/beer/profile/1252/48457/-Quilmes Bock
https://www.beeradvocate.com/beer/profile/37509/144485/-Sigi's Wild Horse
https://www.beeradvocate.com/beer/profile/17033/244106/-Silver Star Bock
https://www.beeradvocate.com/beer/profile/507/19206/-Gulpener Lentebock
https://www.beeradvocate.com/beer/profile/2097/125977/-Summer Wonder
https://www.beeradvocate.com/beer/profile/6052/27676/-Innstadt D' Inn'Staade
https://www.beeradvocate.com/beer/profile/933/46725/-Stolichno Bock
https://www.beeradvocate.com/beer/profile/16434/81930/-'na Biretta Rossa
https://www.beeradvocate.com/beer/profile/403/90062/-Honey Bock
https://www.

In [42]:
print(beerlist)

[{'name': 'Bock', 'brewery': 'Bull Falls Brewery', 'location': 'Wisconsin', 'location2': 'United States', 'style': 'Bock', 'ba_score': '87', 'over_all_rank': 'Ranked #17,576', 'style_rank': 'Ranked #32', 'alcohol_percent': '7.2%', 'avg_rating': '3.85', 'percent_dev': '10.65%', 'review_count': '10', 'rating_count': '26', 'status': 'Active', 'date_added': 'Apr 04, 2008'}, {'name': 'Pfungstädter St. Nikolaus', 'brewery': 'Pfungstädter Brauerei', 'location': 'Germany', 'location2': 'Na', 'style': 'Bock', 'ba_score': '87', 'over_all_rank': 'Ranked #16,163', 'style_rank': 'Ranked #25', 'alcohol_percent': '6.9%', 'avg_rating': '3.89', 'percent_dev': '7.71%', 'review_count': '15', 'rating_count': '25', 'status': 'Active', 'date_added': 'Dec 04, 2004'}, {'name': 'San Martino La Gold', 'brewery': 'Birrificio Ticinese', 'location': 'Switzerland', 'location2': 'Na', 'style': 'Bock', 'ba_score': '80', 'over_all_rank': 'Na', 'style_rank': 'Na', 'alcohol_percent': '7.6%', 'avg_rating': '3.41', 'perce

In [43]:
df=pd.DataFrame(beerlist)
df

,name,brewery,location,location2,style,ba_score,over_all_rank,style_rank,alcohol_percent,avg_rating,percent_dev,review_count,rating_count,status,date_added
0,Bock,Bull Falls Brewery,Wisconsin,United States,Bock,87,"Ranked #17,576",Ranked #32,7.2%,3.85,10.65%,10,26,Active,"Apr 04, 2008"
1,Pfungstädter St. Nikolaus,Pfungstädter Brauerei,Germany,Na,Bock,87,"Ranked #16,163",Ranked #25,6.9%,3.89,7.71%,15,25,Active,"Dec 04, 2004"
2,San Martino La Gold,Birrificio Ticinese,Switzerland,Na,Bock,80,Na,Na,7.6%,3.41,14.37%,7,25,Active,"Feb 23, 2013"
3,Bibock,Birrificio Italiano,Italy,Na,Bock,86,"Ranked #18,250",Ranked #34,6.2%,3.83,10.97%,13,25,Active,"Jun 12, 2005"
4,Bourbon Barrel Bock,Weldebräu,Germany,Na,Bock,84,Na,Na,6.6%,3.63,14.6%,10,25,Active,"Mar 02, 2008"
5,Quilmes Bock,Cervecería y Maltería Quilmes / Grupo Bemberg,Argentina,Na,Bock,72,"Ranked #28,158",Ranked #99,6.3%,2.81,23.13%,9,23,Active,"Mar 14, 2009"
6,Sigi's Wild Horse,Tivoli Brewing Company,Colorado,United States,Bock,83,Na,Na,6.7%,3.6,14.72%,9,23,Active,"Nov 06, 2014"
7,Silver Star Bock,Franconia Brewing Company,Texas,United States,Bock,83,Na,Na,5.06%,3.58,13.13%,3,23,Active,"Aug 10, 2016"
8,Gulpener Lentebock,Gulpener Bierbrouwerij B.V.,Netherlands,Na,Bock,81,Na,Na,6.5%,3.43,9.91%,8,22,Active,"Apr 12, 2005"
9,Summer Wonder,Kuhnhenn Brewing Company,Michigan,United States,Bock,92,Na,Na,13%,4.24,6.84%,1,22,Active,"Jun 21, 2014"


In [45]:
len(beerlinks)

50

##### Building together code to loop through links of different styles of beer (zooming out another level/ another loop)
-challanges: url for different beer styles has a slight change in on of the parameters for each style of beer, need to be mindful of this in loop, meaing a few changes must be implimented on code above


In [ ]:
# root='https://www.beeradvocate.com'
# header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 

    
# website=f'{root}/beer/styles/'
# scraper = cloudscraper.create_scraper() 
# info = scraper.get(website)  
# soup = BeautifulSoup(info.text, 'lxml')
# container=soup.find('div',{'id':'ba-content'})
# container.find_all('a', href=True)[0:-1]

# style_links=[]
# for link in container.find_all('a', href=True)[0:-1]:
#     style_links.append(link['href'])

#  beerlinks=[]
#         for link in table.find_all("a", href=re.compile("profile"))[::2]:
#             beerlinks.append(root + link['href'])

In [ ]:
# style_links

In [105]:
root='https://www.beeradvocate.com'
header= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 

website=f'{root}/beer/styles/'
scraper = cloudscraper.create_scraper() 
info = scraper.get(website)  
soup = BeautifulSoup(info.text, 'lxml')
container=soup.find('div',{'id':'ba-content'})

beerlist=[]

style_href=[]
for link in container.find_all('a', href=True)[0:-1]:
    style_href.append(link['href'])

for h in style_href:
    webber=f'{root}{h}'
    time.sleep(5)
    print(webber)
    

#     for x in range(0,100000,50):
#         print(requests.get(f'{root}/beer/styles/32/?sort=revsD&start={x}'))
#         if requests.get(f'{root}/beer/styles/32/?sort=revsD&start={x}').status_code == 200:    
#             web=f'{webber}?sort=revsD&start={x}'
#             scraper = cloudscraper.create_scraper() 
#             info = scraper.get(web)  
#             soup = BeautifulSoup(info.text, 'lxml')
#             table=soup.find('table')

#             beerlinks=[]
#             for link in table.find_all("a", href=re.compile("profile"))[::2]:
#                 beerlinks.append(root + link['href'])


#             for link in beerlinks:
#                 r=requests.get(link, headers=header)
#                 soup=BeautifulSoup(r.content,'lxml')
#                 name=soup.h1.find(string=True)
#                 try:    
#                     brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
#                 except:
#                     brewery= 'Na'
#                 try:
#                     location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
#                 except:
#                     location= 'Na'
#                 try:
#                     location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
#                 except:
#                     location2= 'Na'
#                 try:
#                     style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
#                 except:
#                     style= 'Na'
#                 try:
#                     ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
#                 except:
#                     ba_score= 'Na'
#                 try:
#                     over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
#                 except:
#                     over_all_rank= 'Na'
#                 try:
#                     style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
#                 except:
#                     style_rank= 'Na'
#                 try:
#                     alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
#                 except:
#                     alcohol_percent= 'Na'
#                 try:
#                     avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
#                 except:
#                     avg_rating= 'Na'
#                 try:
#                     percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
#                 except:
#                     percent_dev= 'Na'
#                 try:
#                     review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
#                 except:
#                     review_count= 'Na'
#                 try:
#                     rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
#                 except:
#                     rating_count= 'Na'
#                 try:
#                     status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
#                 except:
#                     status= 'Na'
#                 try:
#                     date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
#                 except:
#                     date_added= 'Na'
#                 beer_info= {
#                              'name': name,
#                              'brewery': brewery,
#                             'location':location,
#                             'location2':location2,
#                             'style': style,
#                             'ba_score':ba_score,
#                             'over_all_rank': over_all_rank,
#                             'style_rank': style_rank,
#                             'alcohol_percent': alcohol_percent,
#                             'avg_rating':avg_rating,
#                             'percent_dev': percent_dev,
#                             'review_count': review_count,
#                             'rating_count':rating_count,
#                             'status': status,
#                             'date_added': date_added
#                             }
#                 beerlist.append(beer_info)
#                 print(f'{h}-{x}-{link}-{name}')
#         else:
#             break          



https://www.beeradvocate.com/beer/styles/32/
https://www.beeradvocate.com/beer/styles/35/
https://www.beeradvocate.com/beer/styles/36/
https://www.beeradvocate.com/beer/styles/33/
https://www.beeradvocate.com/beer/styles/92/
https://www.beeradvocate.com/beer/styles/86/
https://www.beeradvocate.com/beer/styles/73/
https://www.beeradvocate.com/beer/styles/119/
https://www.beeradvocate.com/beer/styles/74/
https://www.beeradvocate.com/beer/styles/75/
https://www.beeradvocate.com/beer/styles/57/
https://www.beeradvocate.com/beer/styles/18/
https://www.beeradvocate.com/beer/styles/68/
https://www.beeradvocate.com/beer/styles/47/
https://www.beeradvocate.com/beer/styles/147/
https://www.beeradvocate.com/beer/styles/238/
https://www.beeradvocate.com/beer/styles/240/
https://www.beeradvocate.com/beer/styles/149/
https://www.beeradvocate.com/beer/styles/29/
https://www.beeradvocate.com/beer/styles/46/
https://www.beeradvocate.com/beer/styles/7/
https://www.beeradvocate.com/beer/styles/31/
https:

KeyboardInterrupt: 

In [ ]:
# root='https://www.beeradvocate.com'  #base url
# headers= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'} 
# website=f'{root}/beer/styles/'  #base url + path to beer style list 
# scraper = cloudscraper.create_scraper() #calling a scraper session
# response = scraper.get(website,headers=headers)  #built scraper (documnetation says treat it like response)
# soup = BeautifulSoup(response.text, 'html.parser')  #soup of beer style list 
# container=soup.find('div',{'id':'ba-content'}) #getting section of html that contains most of href style ids'
# # print(response.status_code)    ##ran and got 200 scraper works

# beerlist=[]

style_href=[link['href'] for link in  container.find_all('a', href=True)][0:-1] #getting text of href that needs to later be paired with root and other parameters
# print(len(style_href)) ## 120 links from beer style list
for h in style_href: #contatinating root and h to pull up style lists
#     webber=f'{root}{h}'
#     print(f'{webber}')
for x in range(0,100000,50):
    m=f'{root}{h}?sort=revsD&start={x}'
    print(m)
#     scraper = cloudscraper.create_scraper()
    res = scraper.get(webber,headers=headers)
    time.sleep(3)
    print(scraper.get(f'{res}').status_code)
    print(response.status_code) ##ran 200 for each response from webber
    if scraper.get(webber).status_code == 200:   
        scraper = cloudscraper.create_scraper()
        response = scraper.get(webber,headers=headers)
#         time.sleep(3)
#         print(response.status_code)
        print(f'{webber}')
        for x in range(0,100000,50):
            web=f'{webber}{x}'
            if scraper.get(f'{webber}{x}').status_code == 200:        
                scraper = cloudscraper.create_scraper() 
                info = scraper.get(website)  
                soup = BeautifulSoup(info.text, 'lxml')
                table=soup.find('table')
                beerlinks=[root + link['href'] for link in table.find_all("a", href=re.compile("profile"))][::2]
                for link in beerlinks:
                    scraper = cloudscraper.create_scraper()
                    r=scraper.get(link, headers=header)
                    soup=BeautifulSoup(r.text,'lxml')
                    name=soup.h1.find(string=True)
#                     try:    
#                         brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
#                     except:
#                         brewery= 'Na'
#                     try:
#                         location=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
#                     except:
#                         location= 'Na'
#                     try:
#                         location2=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
#                     except:
#                         location2= 'Na'
#                     try:
#                         style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
#                     except:
#                         style= 'Na'
#                     try:
#                         ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
#                     except:
#                         ba_score= 'Na'
#                     try:
#                         over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
#                     except:
#                         over_all_rank= 'Na'
#                     try:
#                         style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
#                     except:
#                         style_rank= 'Na'
#                     try:
#                         alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
#                     except:
#                         alcohol_percent= 'Na'
#                     try:
#                         avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
#                     except:
#                         avg_rating= 'Na'
#                     try:
#                         percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
#                     except:
#                         percent_dev= 'Na'
#                     try:
#                         review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
#                     except:
#                         review_count= 'Na'
#                     try:
#                         rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
#                     except:
#                         rating_count= 'Na'
#                     try:
#                         status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
#                     except:
#                         status= 'Na'
#                     try:
#                         date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
#                     except:
#                         date_added= 'Na'
#                     beer_info= {
#                                  'name': name,
#                                  'brewery': brewery,
#                                 'location':location,
#                                 'location2':location2,
#                                 'style': style,
#                                 'ba_score':ba_score,
#                                 'over_all_rank': over_all_rank,
#                                 'style_rank': style_rank,
#                                 'alcohol_percent': alcohol_percent,
#                                 'avg_rating':avg_rating,
#                                 'percent_dev': percent_dev,
#                                 'review_count': review_count,
#                                 'rating_count':rating_count,
#                                 'status': status,
#                                 'date_added': date_added
#                                 }
#                     beerlist.append(beer_info)
#                     print(f'{h}-{x}-{link}-{name}')
#             else:
#                 break 


In [88]:
def cook_soup(url):
    scraper = cloudscraper.create_scraper()
    headers= {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}
    response = scraper.get(url,headers=headers)
    soup = BeautifulSoup(response.text,'lxml')
    print(f'url: {url} - status:{response.status_code}-')
    

#will create an instance of cloudscraper session for url and print status code
# https://stackoverflow.com/questions/73027049/how-to-ensure-response-200-for-multiple-requests-when-webscraping

In [98]:
test=cook_soup('https://www.beeradvocate.com/beer/profile/57/7714/')


url: https://www.beeradvocate.com/beer/profile/57/7714/ - status:200-


In [99]:
url="https://www.beeradvocate.com/beer/profile/417/47795/"
test_2=cook_soup(url)

url: https://www.beeradvocate.com/beer/profile/417/47795/ - status:200-


In [ ]:
root='https://www.beeradvocate.com' # base url
website=f'{root}/beer/styles/' #url that has links to styles of beers (14 styles, and 120 links)
cook_soup(website) #makes soup of previous url
container=soup.find('div',{'id':'ba-content'}) #looks for place where href are
style_href=[link['href'] for link in  container.find_all('a', href=True)][0:-1] #saves hrefs
for h in style_href:
    webber=f'{root}{h}'
#     print(webber)
    cook_soup(webber)
    for x in range(0,100000,50):
        web=f'{root}{h}?sort=revsD&start={x}'
        cook_soup(web)
        if response.status_code != 200:
            break ##
    
        